In [1]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 1.0 MB/s eta 0:00:0000:0100:01m


In [15]:
import redis
r = redis.Redis(host='redis', port=6379, decode_responses=True)

In [16]:
r.ping()

True

In [17]:
n_actors = len([k for k in r.scan_iter("actor:*")])
n_movies = len([k for k in r.scan_iter("movie:*")])
print("Nombre d'acteurs :", n_actors)
print("Nombre de films :", n_movies)

Nombre d'acteurs : 1320
Nombre de films : 922


In [18]:
actors = []
for key in r.scan_iter("actor:*"):
    dob = int(r.hget(key, "date_of_birth"))
    if dob < 1980:
        actors.append(r.hgetall(key))
    if len(actors) >= 5:
        break
actors


[{'first_name': 'Wesley', 'last_name': 'Snipes', 'date_of_birth': '1962'},
 {'first_name': 'Tommy', 'last_name': 'Lee Jones', 'date_of_birth': '1946'},
 {'first_name': 'Andrew', 'last_name': 'Rannells', 'date_of_birth': '1978'},
 {'first_name': 'Mahershala', 'last_name': 'Ali', 'date_of_birth': '1974'},
 {'first_name': 'Andrew', 'last_name': 'Scott', 'date_of_birth': '1976'}]

In [19]:

for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Imitation Game":
        genre = r.hget(key, "genre")
        rating = r.hget(key, "rating")
        print(f"Genre : {genre} | Note : {rating}")
        break

Genre : Biography | Note : 8.5


In [20]:

top_movies = sorted(
    [(r.hget(k, "title"), float(r.hget(k, "rating") or 0))
     for k in r.scan_iter("movie:*")],
    key=lambda x: x[1], reverse=True
)[:5]
top_movies


[('Boy 9', 9.4),
 ('Vegas (doc)', 9.4),
 ('The Shawshank Redemption', 9.3),
 ('Ween Live in Chicago', 9.2),
 ('Over Canada: An Aerial Adventure', 9.1)]

In [21]:
high_rating_count = sum(
    1 for k in r.scan_iter("movie:*")
    if float(r.hget(k, "rating") or 0) > 7.5
)
print("Nombre de films avec note > 7.5 :", high_rating_count)


Nombre de films avec note > 7.5 : 183


In [22]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Imitation Game":
        r.hset(key, "rating", 8.5)
        print("Mise à jour réussie.")
        break


Mise à jour réussie.


In [23]:

new_id = max(int(k.split(":")[1]) for k in r.scan_iter("actor:*")) + 1
r.hset(f"actor:{new_id}", mapping={"first_name": "Zendaya", "last_name": "", "date_of_birth": 1996})
print("Acteur ajouté avec ID :", new_id)


Acteur ajouté avec ID : 1321


In [24]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Room":
        r.delete(key)
        print("Film supprimé :", key)
        break
